In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
import psycopg2
con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
port= '5439', user= 'mantix', password= 'Mantix123!')
cur = con.cursor()

In [3]:
query = """(SELECT DISTINCT customer_account_id,
                               DATEDIFF(DAY,MAX(submitted_date),'2022-02-16') AS days_since_last_transaction,
                               COUNT(*) AS toal_purchases,
                               SUM(total) AS total_amount
                        FROM cdp.orders_data
                        WHERE tenant_id = 'TNB00001'
                        AND   store_id = 'STOM000000001'
                        AND    status NOT IN ('Abandoned','Errored','Pending','PendingReview','Cancelled','null')
                        GROUP BY customer_account_id)"""

In [4]:
b = pd.read_sql(query,con)

In [5]:
b

,customer_account_id,days_since_last_transaction,toal_purchases,total_amount
0,2b59397433c144e3b6c05cf0f1c2fcb8,190,1,37.48
1,467da31318cd46b78bfc56eab4272b53,-22,1,18.01
2,051b1d2c86c84fbfb676cba15319982e,55,1,47.98
3,4851cf77aa8d4c57a3708691c6abdb67,-105,8,478.30
4,3eb78bebf69240aabe2707d89c7c31dd,-95,2,757.87
...,...,...,...,...
17353,53b508a94de2467481e7c273eb336686,31,1,21.15
17354,cc3b3ad4f8e64f7eb2bd0397bbbce3d9,9,1,53.91
17355,b681fd7fc40e43d893b148a3e1735352,3,1,267.29
17356,bc7cd4f52779446880f63af9cf511e0f,-8,1,69.98


In [6]:
b['total_amount'] = b['total_amount'].fillna(0)

In [7]:
row_indexes=b[b['days_since_last_transaction']>=90].index
b.loc[row_indexes,'churn']= 1


In [8]:
b['churn'] = b['churn'].fillna(0)

In [9]:
b.churn.value_counts()

0.0    13009
1.0     4349
Name: churn, dtype: int64

In [10]:
b['churn'] = b['churn'].astype('category')

In [11]:
response = pd.DataFrame()
response['count'] = b.churn.value_counts()
response['percentage'] = (response['count'] / response['count'].sum()) * 100
print(response)

     count  percentage
0.0  13009    74.94527
1.0   4349    25.05473


In [12]:
b.shape

(17358, 5)

In [13]:
b = b.dropna()

In [14]:
b.shape

(17358, 5)

In [15]:
b.head()

,customer_account_id,days_since_last_transaction,toal_purchases,total_amount,churn
0,2b59397433c144e3b6c05cf0f1c2fcb8,190,1,37.48,1.0
1,467da31318cd46b78bfc56eab4272b53,-22,1,18.01,0.0
2,051b1d2c86c84fbfb676cba15319982e,55,1,47.98,0.0
3,4851cf77aa8d4c57a3708691c6abdb67,-105,8,478.30,0.0
4,3eb78bebf69240aabe2707d89c7c31dd,-95,2,757.87,0.0


In [16]:
#setting our training and test set
training_set = b.drop(['churn','days_since_last_transaction','customer_account_id'], axis=1)
target_var = b['churn'].values

In [17]:
training_set.shape

(17358, 2)

In [18]:
#splitting our data...
from sklearn.model_selection import train_test_split
xtrain,xtest, ytrain, ytest = train_test_split(training_set, target_var, test_size=0.3, random_state=0)

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report
gbr = GradientBoostingClassifier(n_estimators = 500)
gbr.fit(xtrain,ytrain)
prediction_gbr = gbr.predict(xtest)
print("Training accuracy: %.2f" %(gbr.score(xtrain, ytrain) * 100))
print("Test accuracy    : %.2f" %(gbr.score(xtest, ytest) * 100))
print("")
print("Confusion matrix..")
print(confusion_matrix(ytest, prediction_gbr))
print("")
print(classification_report(ytest, prediction_gbr))

Training accuracy: 76.69
Test accuracy    : 74.31

Confusion matrix..
[[3825   54]
 [1284   45]]

              precision    recall  f1-score   support

         0.0       0.75      0.99      0.85      3879
         1.0       0.45      0.03      0.06      1329

    accuracy                           0.74      5208
   macro avg       0.60      0.51      0.46      5208
weighted avg       0.67      0.74      0.65      5208



In [20]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 500)
random_forest.fit(xtrain,ytrain)
prediction_rf = random_forest.predict_proba(xtest)
print("Training accuracy: %.2f" %(random_forest.score(xtrain, ytrain) * 100))
print("Test accuracy    : %.2f" %(random_forest.score(xtest, ytest) * 100))
print("Test RMSE        : %.2f" %(np.sqrt(mean_squared_error(ytest, prediction_rf))))
print("")
print("Confusion matrix..")
print(confusion_matrix(ytest, prediction_rf))
print("")
print(classification_report(ytest, prediction_rf))

Training accuracy: 83.98
Test accuracy    : 70.22


ValueError: y_true and y_pred have different number of output (1!=2)

In [21]:
import joblib
joblib.dump(random_forest, 'Bimart_churn.pkl') 
rf_prop = joblib.load('Bimart_churn.pkl') 

In [22]:
prediction_rf = random_forest.predict_proba(training_set)

In [23]:
prediction_rf_df = pd.DataFrame(data = prediction_rf)

In [24]:
prediction_rf_df.head()

,0,1
0,0.673491,0.326509
1,0.995919,0.004081
2,1.000000,0.000000
3,1.000000,0.000000
4,0.994000,0.006000


In [25]:
prediction_rf_df = prediction_rf_df.rename(columns={prediction_rf_df.columns[1]: 'churn_prediction'})

In [26]:
b.shape

(17358, 5)

In [66]:
b['churn_prediction'] = prediction_rf_df.iloc[1]

In [67]:
b.head()

,customer_account_id,days_since_last_transaction,toal_purchases,total_amount,churn,churn_prediction
0,a27edc0cf0c54dd4bc6d723ef9d2358a,-197,9,281.61,0.0,0.9044
1,ffcd62266a654613907e2751c0b71938,-76,1,47.71,0.0,0.0956
2,817aaae2d0a44ad894c2ac8ac1cdf5c4,-13,1,5.00,0.0,NaN
3,8bf8c7fc40d34d1b818dc0bb2f5ffb20,-193,11,831.45,0.0,NaN
4,3912ff1f397e466d9af473be9ccf3885,94,1,5.00,1.0,NaN


In [28]:
prediction_rf_df['customer_account_id'] = prediction_rf_df.iloc[0]

In [29]:
prediction_rf_df.head()

,0,churn_prediction,customer_account_id
0,0.673491,0.326509,0.673491
1,0.995919,0.004081,NaN
2,1.000000,0.000000,NaN
3,1.000000,0.000000,NaN
4,0.994000,0.006000,NaN


In [30]:
new_df = b.merge(prediction_rf_df, left_index=True, right_index=True, how='inner')

In [32]:
results_df = new_df[['customer_account_id_x','churn_prediction']]

In [33]:
results_df = results_df.rename(columns={'customer_account_id_x': 'customer_account_id'})

In [34]:
results_df.head()

,customer_account_id,churn_prediction
0,2b59397433c144e3b6c05cf0f1c2fcb8,0.326509
1,467da31318cd46b78bfc56eab4272b53,0.004081
2,051b1d2c86c84fbfb676cba15319982e,0.000000
3,4851cf77aa8d4c57a3708691c6abdb67,0.000000
4,3eb78bebf69240aabe2707d89c7c31dd,0.006000


In [37]:
results_df['churn_prediction'] = results_df['churn_prediction'].mul(100).round(2)

In [38]:
results_df.head()

,customer_account_id,churn_prediction
0,2b59397433c144e3b6c05cf0f1c2fcb8,32.65
1,467da31318cd46b78bfc56eab4272b53,0.41
2,051b1d2c86c84fbfb676cba15319982e,0.00
3,4851cf77aa8d4c57a3708691c6abdb67,0.00
4,3eb78bebf69240aabe2707d89c7c31dd,0.60


In [39]:
results_df['tenant_id'] = 'TNB00001'
results_df['store_id'] = 'STOM000000001'

In [40]:
results_df.head()

,customer_account_id,churn_prediction,tenant_id,store_id
0,2b59397433c144e3b6c05cf0f1c2fcb8,32.65,TNB00001,STOM000000001
1,467da31318cd46b78bfc56eab4272b53,0.41,TNB00001,STOM000000001
2,051b1d2c86c84fbfb676cba15319982e,0.00,TNB00001,STOM000000001
3,4851cf77aa8d4c57a3708691c6abdb67,0.00,TNB00001,STOM000000001
4,3eb78bebf69240aabe2707d89c7c31dd,0.60,TNB00001,STOM000000001


In [51]:
def load_db(results_df):
    con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
    port= '5439', user= 'mantix', password= 'Mantix123!')
    cur = con.cursor()

    def drop_table(conn,table):
        try:
            query = f"""DROP TABLE IF EXISTS {table}"""
            cur = conn.cursor()
            cur.execute(query)
            conn.commit()
            print(f'Drop table {table} successfully')
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cur.close()
            #return 1

    def create_table(conn,table):
        try:
            query_col = """(customer_account_id VARCHAR(1000),
                churn_prediction FLOAT,tenant_id VARCHAR(1000),store_id VARCHAR(1000))"""
            query = f"""
                    CREATE TABLE {table} 
                    """+query_col
            cur = conn.cursor()
            cur.execute("ROLLBACK")
            conn.commit()
            cur.execute(query)
            conn.commit()
            print(f'Successfully created the table {table}')
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cur.close()
            return 1
            
    def single_insert(conn, insert_req):
        """ Execute a single INSERT request """
        cursor = conn.cursor()
        try:
            cursor.execute(insert_req)
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        cursor.close()
        
    def insert_dataframe(results_df,conn,table):
        for i in results_df.index:
            query = f"""
                    INSERT into {table}(customer_account_id,churn_prediction,tenant_id,store_id) 
                        values('%s',%s,%s,%s);
                    """ % (results_df.loc[i]['customer_account_id'], 
                        results_df.loc[i]['churn_prediction'],
                         "'"+str(results_df.loc[i]['tenant_id'])+"'",
                        "'"+str(results_df.loc[i]['store_id'])+"'")
            if i%50==0:
                print(i,end=' ')
            single_insert(conn, query)
        conn.commit()
        print('\nSuccessfully inserted dataframe into the table')
        
    table = 'cdp.churn'
    drop_table(con,table)
    create_table(con,table)
    insert_dataframe(results_df,con,table)

if __name__ == '__main__':


    # load data into database
    load_db(results_df)



Drop table cdp.churn successfully
Successfully created the table cdp.churn
0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3400 3450 3500 3550 3600 3650 3700 3750 3800 3850 3900 3950 4000 4050 4100 4150 4200 4250 4300 4350 4400 4450 4500 4550 4600 4650 4700 4750 4800 4850 4900 4950 5000 5050 5100 5150 5200 5250 5300 5350 5400 5450 5500 5550 5600 5650 5700 5750 5800 5850 5900 5950 6000 6050 6100 6150 6200 6250 6300 6350 6400 6450 6500 6550 6600 6650 6700 6750 6800 6850 6900 6950 7000 7050 7100 7150 7200 7250 7300 7350 7400 7450 7500 7550 7600 7650 7700 7750 7800 7850 7900 Error: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.



InterfaceError: connection already closed

In [44]:
drop_table(con,table)
create_table(con,table)
insert_dataframe(cltv_final,con,table)

NameError: name 'drop_table' is not defined

In [ ]:
def load_db(cltv_final):
    con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
    port= '5439', user= 'mantix', password= 'Mantix123!')
    cur = con.cursor()

    def drop_table(conn,table):
        try:
            query = f"""DROP TABLE IF EXISTS {table}"""
            cur = conn.cursor()
            cur.execute(query)
            conn.commit()
            print(f'Drop table {table} successfully')
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cur.close()
            #return 1

    def create_table(conn,table):
        try:
            query_col = """(customer_account_id VARCHAR(1000),
                cltv_6M FLOAT,cltv_12M FLOAT,
                cltv_24M FLOAT,tenant_id VARCHAR(1000),store_id VARCHAR(1000),create_dms VARCHAR(1000))"""
            query = f"""
                    CREATE TABLE {table} 
                    """+query_col
            cur = conn.cursor()
            cur.execute("ROLLBACK")
            conn.commit()
            cur.execute(query)
            conn.commit()
            print(f'Successfully created the table {table}')
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cur.close()
            return 1
            
    def single_insert(conn, insert_req):
        """ Execute a single INSERT request """
        cursor = conn.cursor()
        try:
            cursor.execute(insert_req)
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        cursor.close()
        
    def insert_dataframe(cltv_final,conn,table):
        for i in cltv_final.index:
            query = f"""
                    INSERT into {table}(customer_account_id,cltv_6M,cltv_12M,cltv_24M,tenant_id,store_id,create_dms) 
                        values('%s',%s,%s,%s,%s,%s,%s);
                    """ % (cltv_final.loc[i]['CustomerId'], 
                        cltv_final.loc[i]['cltv_6M'],
                        
                        cltv_final.loc[i]['cltv_12M'],
                        
                        cltv_final.loc[i]['cltv_24M'],
                        
                        "'"+str(cltv_final.loc[i]['tenant_id'])+"'",
                        "'"+str(cltv_final.loc[i]['store_id'])+"'",
                         "'"+str(cltv_final.loc[i]['create_dms'])+"'")
            if i%50==0:
                print(i,end=' ')
            single_insert(conn, query)
        conn.commit()
        print('\nSuccessfully inserted dataframe into the table')
        
    table = 'cdp.cltv'
    drop_table(con,table)
    create_table(con,table)
    insert_dataframe(cltv_final,con,table)

if __name__ == '__main__':

    # start cltv
    cltv_final = start_cltv()

    # load data into database
    load_db(cltv_final)
